In [ ]:
import ipyvuetify as v
import ipywidgets as widgets
import qgrid
import numpy as np
import pandas as pd
import pandas_schema as ps
from pandas_schema.validation import InListValidation, IsDtypeValidation
import json
from io import BytesIO
import os
from os.path import join, exists

file_name = "geo_data.csv"
file_path = join(os.getcwd(),"datasets",file_name)
DATA_EXISTS = exists(file_path)

grid_options = {
    # SlickGrid options
    'fullWidthRows': True,
    'syncColumnCellResize': True,
    'forceFitColumns': False,
    'defaultColumnWidth': 150,
    'rowHeight': 28,
    'enableColumnReorder': False,
    'enableTextSelectionOnCells': True,
    'editable': False,
    'autoEdit': False,
    'explicitInitialization': True,

    # Qgrid options
    'maxVisibleRows': 15,
    'minVisibleRows': 8,
    'sortable': True,
    'filterable': False,
    'highlightSelectedCell': False,
    'highlightSelectedRow': True
}

# Esquema de validación
schema = ps.Schema([
    ps.Column(
        name = "nombre",
        validations = [
            IsDtypeValidation(dtype=np.object),
        ],
        allow_empty = False,
    ),
    ps.Column(
        name = "x",
        validations = [
            IsDtypeValidation(dtype=np.floating),
        ],
        allow_empty = False,
    ),
    ps.Column(
        name = "y",
        validations = [
            IsDtypeValidation(dtype=np.floating),
        ],
        allow_empty = False,
    ),
    ps.Column(
        name = "utm",
        validations = [
            IsDtypeValidation(dtype=np.object),
            InListValidation(
                options = ["19K", "20K"],
                case_sensitive = False,
            )
        ],
        allow_empty = False,
    ),
])

# Función de apoyo: JSON a Vuetify Tree Viewer

id_counter = 0

def replace_lists(x):
    if type(x) == dict:
        return({k: replace_lists(v) for k,v in x.items()})
    if type(x) == list:
        return({i: replace_lists(element) for i,element in enumerate(x)})
    else:
        return(x)
    
def dict_to_tree(x):
    global id_counter
    tree = []
    if type(x) == dict:
        list_free_x = replace_lists(x)
        tree = dict_to_tree_h(list_free_x)
    id_counter = 0
    return tree
    
def dict_to_tree_h(d):
    global id_counter
    tree = []
    for k,v in d.items():
        id_counter += 1
        entry = {"id": id_counter}
        if type(v) == dict:
            entry["name"] = str(k)
            entry["children"] = dict_to_tree_h(v)
        else:
            entry["name"] = f"{str(k)}: {str(v)}"
        tree.append(entry)
    return tree

def p(text):
    return v.Html(tag="p", children=[text])
def b(text):
    return v.Html(tag="b", children=[text])
def vue_list(pairs):
    v_list = v.List(
        two_line = True, 
        dense = True,
        children = [v.ListItemGroup(color="primary", children=[
            v.ListItem(children=[
                v.ListItemContent(children=[
                    v.ListItemTitle(children=[pair[0]]),
                    v.ListItemSubtitle(children=[pair[1]]),
                ])
            ])
        for pair in pairs])],
    )
    return(v_list)

In [ ]:
upload_widget = widgets.FileUpload(
    accept = ".csv",
    multiple = False,
    button_style = "info",
)
upload_widget.add_class("v-btn")
upload_widget.add_class("primary")
upload_widget.add_class("ripple")
upload_widget.add_class("v-btn--block")

sheet_info = v.Container(children=["Ninguna planilla subida."])

grid = qgrid.QGridWidget(
    df = pd.read_csv(file_path) if DATA_EXISTS else pd.DataFrame(),
    grid_options = grid_options,
)

restore_button = widgets.Button(
    description = "Reemplazar",
    disabled = True,
    button_style = "success",
    tooltip = 'Reemplazar la planilla de datos para la aplicación.',
    icon = 'archive',
)
restore_button.add_class("v-btn")
restore_button.add_class("secondary")
restore_button.add_class("ripple")
restore_button.add_class("v-btn--block")

restore_info = v.Container(
    children = ["Planilla base encontrada." if DATA_EXISTS else "No se encontró una planilla base."],
)

def on_upload_widget_value_change(change):
    # Limpeza de widgets
    grid.df = pd.DataFrame()
    restore_button.disabled = True
    
    # Recuperación de datos del widgets de archivo
    file_val = list(change["new"].values())[0]
    metadata = file_val["metadata"]
    content = file_val["content"]
    
    # Validación: "Archivo es CSV"
    if metadata["name"][-4:] != ".csv":
        sheet_info.children = [
            b("Archivo inválido:"),
            p("El archivo debe ser del tipo CSV."),
        ]
        return
    
    # Lectura del archivo
    input_df = pd.read_csv(BytesIO(content))
    
    # Validación: "Planilla tiene el esquema correcto"
    errors = schema.validate(input_df)
    if len(errors) > 0:
        restore_info.children = ["Planilla no reemplazada."]
        sheet_info.children = [
            b("Archivo inválido:"),
            p("Errores de validación:"),
            vue_list([(f"Error {i+1}",str(e)) for i,e in enumerate(errors)]),
        ]
        return
    
    # Archivo Válido. Mostrar detalles
    sheet_info.children = [
        b("Archivo válido:"),
        p("Detalles:"),
        vue_list([(str(k),str(v)) for k,v in metadata.items()]),
    ]
    restore_info.children = ["Planilla subida (todavía no reemplazada)."]
    grid.df = input_df
    restore_button.disabled = False
    
def on_restore_button_click(button):
    grid.df.to_csv(file_path)
    restore_info.children = [
        b("Planilla reemplazada."),
        v.Icon(children=["mdi-check-circle"]),
    ]
    restore_button.disabled=True

upload_widget.observe(on_upload_widget_value_change, names="value")
restore_button.on_click(on_restore_button_click)

In [ ]:
instructions = v.Container(children=[
    v.Html(tag="h1",children=["Restaurar Planilla de Datos Geográficos"]),
    "Esta aplicación permite subir, validar y restaurar la planilla de datos geográficos, "
    "que la aplicación de licencias y registros usará de manera preferencial."
    "El archivo deber ser de formato 'CSV' y contar con las siguientes columnas:",
    v.Html(tag="br"),
    v.Divider(),
    v.Card(tile=True, children=[
        vue_list([
            ("nombre (str)", "Nombre de prueba."),
            ("x (float)", "Coordenada X."),
            ("y (float)", "Coordenada Y."),
            ("utm (str)", "Zona UTM."),
        ]),
    ]),
    v.Html(tag="br"),
    "De ser valida la planilla subida, reemplazará al anterior y será usado por la aplicación de manera preferencial."
    "Si la aplicación no encuentra los datos geográficos que busca en este archivo, hará uso de los datos obtenidos desde el SIIRAyS.",
    v.Html(tag="br"),
    v.Html(tag="h2", children=["Instrucciones"]),
    v.Html(tag="br"),
    v.Stepper(children=[
        v.StepperHeader(children=[
            v.StepperStep(editable=True, step="1", children=["Ver planilla actual"]),
            v.Divider(),
            v.StepperStep(editable=True, step="2", children=["Subir planilla"]),
            v.Divider(),
            v.StepperStep(editable=True, step="3", children=["Reemplazar planilla"]),
        ]),
        v.StepperItems(children=[
            v.StepperContent(step="1", children=[
                v.Card(hover=True, children=[
                    "La tabla en la sección 'PLANILLA' muestra el archivo usado actualmente por la aplicación.",
                ]),
            ]),
            v.StepperContent(step="2", children=[
                v.Card(hover=True, children=[
                    "Al presionar el botón 'Upload (0)' que se encuentra en la barra lateral de controles, "
                    "la aplicación te permitirá seleccionar un archivo en formato CSV de tu sistema local.",
                    v.Html(tag="br"),
                    "Al abrir el archivo seleccionado este es validado por la aplicación.",
                    v.Html(tag="br"),
                    "Si el archivo pasa el proceso de validación, se mostrarán algunos detalles del archivo y la planilla será actualizada, pero todavía no reemplazará a la versión previa.",
                ]),
            ]),
            v.StepperContent(step="3", children=[
                v.Card(hover=True, children=[
                    "De estar de acuerdo con los datos mostrados, al hacer click en el botón 'Reemplazar' que se encuentra en la barra lateral de controles, "
                    "el archivo usado por las aplicaciones será reemplazado.",
                    v.Html(tag="br"),
                    "Los nuevos datos serán usados por las otras aplicaciones.",
                ]),
            ]),
        ]),
    ]),
    
])

In [ ]:
# Sección: Barra de navegación superior (Selección de widgets)
app_bar = v.Container(
    _metadata = {"mount_id": "content-app-bar"},
    children = [
        v.Tabs(
            v_model = 0,
            background_color = "transparent",
            color = "white",
            children = [
                v.TabsSlider(color="yellow"),
                v.Tab(key="0", children=["General"]),
                v.Tab(key="1", children=["Planilla"]),
            ],
        ),
    ],
)

# Sección: Barra de navegación izquierda (controles)
nav = v.Container(
    _metadata = {"mount_id": "content-nav"},
    fluid = True, 
    children = [
        v.Row(children=[v.ExpansionPanels(
            multiple=True, accordion=True, focusable=True, hover=True,
            children = [
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=["Subir Planilla"]),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[v.Container(pa_1=True, children=[upload_widget])]),
                        v.Divider(),
                        v.Row(children=[sheet_info]),
                    ]),
                ]),
                v.ExpansionPanel(children=[
                    v.ExpansionPanelHeader(children=["Reemplazar Planilla"]),
                    v.ExpansionPanelContent(children=[
                        v.Row(children=[v.Container(pa_1=True,children=[restore_button])]),
                        v.Divider(),
                        v.Row(children=[restore_info]),   
                    ]),
                ]),
            ],
        )]),
    ],
)

# Sección: Principal (Aplicación)
main = v.Container(
    _metadata = {"mount_id": "content-main"},
    children = [
        v.TabsItems(
            v_model = 0,
            children = [
                v.TabItem(key="0", children=[
                    v.Container(fluid=True, children=[instructions])
                ]),
                v.TabItem(key="1", children=[
                    v.Container(fluid=True, children=[grid])
                ]),
            ],
        ),
    ],
)

def on_tab_change(widget, event, data):
    main.children[0].v_model = widget.v_model
    
app_bar.children[0].on_event("change", on_tab_change)

app_bar
nav
main

# treeview = v.Treeview(items=[], activatable=True, hoverable=True, open_on_click=True, rounded=True)